## Imports & helper functions

In [ ]:
%pip install --upgrade py2neo
%pip install --upgrade openai

In [ ]:
from py2neo import Graph


In [ ]:
from itertools import groupby
import random

In [ ]:
import json


def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


def parse_json(json_string):
  json_dict = json.loads(json_string)
  return json_dict


def dict_to_pretty_json(dictionary):
  pretty_json = json.dumps(dictionary, indent=2)
  return pretty_json


def write_dict_to_json(dictionary, file_path):
  with open(file_path, 'w') as json_file:
    json.dump(dictionary, json_file, indent=2)


In [ ]:
import configparser

def read_ini_file(file_path):
    config = configparser.ConfigParser()
    config.read(file_path)
    ini_dict = {section: dict(config.items(section))
                for section in config.sections()}
    return ini_dict


In [ ]:
from openai import OpenAI

In [ ]:
import re


def remove_java_comments(java_source):
    # Regular expression to match Java comments (both single-line and multi-line)
    pattern = r"(//.*?$)|(/\*.*?\*/)"

    # Remove comments using the regular expression
    java_source_without_comments = re.sub(
        pattern, "", java_source, flags=re.MULTILINE | re.DOTALL)

    return java_source_without_comments


def bools(input_string):
    pattern = r'((?:true|false)(?:\s*,\s*(?:true|false))*)'
    match = re.search(pattern, input_string, re.IGNORECASE)

    if match:
        tokens = match.group(1).split(',')
        return [b.strip().lower() == "true" for b in tokens]
    else:
        return []


## Parameters

In [ ]:
# If True: do not call the API, just print the prompts
only_print_prompt = False

In [ ]:
secrets = read_ini_file('secrets.ini')
# project_name = 'jhotdraw'
# project_desc = 'A Java-based drawing framework and application that provides a collection of packages for creating, manipulating, and managing drawings in a graphical user interface.'
project_name = 'k9-mail'
project_desc = 'K-9 Mail is an open source email client for Android focused on making it easy to chew through large volumes of email.'


## Connect to neo4j

To access knowledge graph extracted using javapers which is then loaded to neo4j graph database.

In [ ]:
graph = Graph(secrets['neo4j']['url'], auth=(secrets['neo4j']['username'], secrets['neo4j']['password']))

## Connect to openai

In [ ]:
openai.api_key = secrets['openai']['apikey']
model = "gpt-4"


## Packages to be inspected

In [ ]:
packages = { record['p']['qualifiedName'] for record in graph.run('match (p:Container)-[:contains]->(:Structure) where p.kind="package" return p') }

In [ ]:
len(packages), packages

## Layer definitions

In [ ]:
# layers = [{
#   "name": "Presentation",
#   "responsibility": "This layer is responsible for the user interface and interaction with the outside world.",
#   "characteristics": [
#     "Classes should contain UI elements such as buttons, text fields, labels, etc.",
#     "Event listeners and handlers for user input should be present.",
#     "GUI layout code using Java's UI frameworks (e.g., JavaFX, Swing) is found here.",
#     "User input validation logic can be part of this layer.",
#     "Classes in this layer are concerned with the visual representation of data."]
# },{
#   "name": "Application Services",
#   "responsibility": "Coordinates responses to outside events, orchestrates high-level application logic.",
#   "characteristics": [
#     "Contains classes that manage the flow of the application.",
#     "Coordinates interactions between different components of the application.",
#     "Implements use cases and business logic by invoking methods from the domain layer.",
#     "Manages the sequence of actions in response to user input or system events.",
#     "Often contains classes related to navigation and routing between different parts of the application."]
# },{
#   "name": "Domain Services",
#   "responsibility": "Provides information and services that span the entire domain.",
#   "characteristics": [
#     "Contains classes that encapsulate domain logic and business rules.",
#     "Represents core concepts of the application's problem domain.",
#     "Contains domain-specific data structures and algorithms.",
#     "Classes here should not have direct dependencies on UI or external systems.",
#     "Handles complex domain-specific calculations and validations.",
#     "Business logic that operates on domain objects can be found here."]
# },{
#   "name": "Technical Services",
#   "responsibility": "Provides technical services for connecting to external devices and programs.",
#   "characteristics": [
#     "Contains classes that manage low-level technical aspects such as network communication, file I/O, etc.",
#     "Wraps interactions with external systems, libraries, or APIs.",
#     "Abstracts away technical complexities from the rest of the application.",
#     "Provides utility methods for tasks like data serialization, encryption, etc.",
#     "Typically interfaces with hardware devices, databases, web services, etc."]
# }]

In [ ]:
# layers = [{
#   "name": "Presentation",
#   "responsibility": "This layer is responsible for the user interface and interaction with the outside world.",
#   "characteristics": [
#     "create, initialize, or manage UI elements such as buttons, text fields, labels, etc.",
#     "include event listeners and handlers for user input.",
#     "define the layout of UI components using Java's UI frameworks like JavaFX or Swing.",
#     "perform validation on user input and provide feedback.",
#     "handle the transformation and display of data in a visually meaningful way."]
# },{
#   "name": "Application Services",
#   "responsibility": "Coordinates responses to outside events, orchestrates high-level application logic.",
#   "characteristics": [
#     "manage the overall flow of the application.",
#     "coordinate interactions between different components.",
#     "implement use cases and business logic by invoking domain layer methods.",
#     "manage and execute a sequence of actions in response to events.",
#     "handle navigation and routing between different parts of the application."]
# },{
#   "name": "Domain Services",
#   "responsibility": "Provides information and services that span the entire domain.",
#   "characteristics": [
#     "encapsulate specific domain logic and business rules.",
#     "work with core concepts of the problem domain.",
#     "utilize domain-specific data structures and algorithms.",
#     # "not directly depend on UI or external systems.",
#     "handle complex calculations and validations.",
#     "implement business logic operating on domain objects."]
# },{
#   "name": "Technical Services",
#   "responsibility": "Provides technical services for connecting to external devices and programs.",
#   "characteristics": [
#     "manage low-level technical aspects such as network communication, file I/O, etc.",
#     "wrap interactions with external systems, libraries, or APIs.",
#     "abstract away technical complexities from the rest of the application.",
#     "provide utility for tasks like data serialization, encryption, etc.",
#     "interface with hardware devices, databases, web services, etc."]
# }]

In [ ]:
layers = [{
  'name':'Presentation',
  'responsibility':'is devoted to interfacing with the user',
  'characteristics':[
      "Sets the arrtibutes of UI components, e.g., sets the text of a TextView.",
      "Notifies listeners about user events, such as button clicks or list item selections.",
      "Transforms domain objects into visual representations.",
      "Performs validation on user input."
  ]
},
{
  'name':'Application Services',
  'responsibility':'coordinates responses to outside events and orchestrates the high-level application logic',
  'characteristics':[
      "Reacts to events, e.g., methods with name starting with 'on-' in a class that is called '-Listener'.",
      "Handles application lifecycle events, e.g., saves application state when the app is paused.",
      "Handles application navigation, e.g., switching Activities.",
      "Manages the overall flow of the application."
  ]
},
{
  'name':'Domain Services',
  'responsibility':"provides information and services about the application's problem domain",
  'characteristics':[
      "Returns information about domain objects.",
      "Creates or modifies domain objects.",
      "Performs complex calculation or computation involving domain objects.",
      "Manages a collection of or relationship between domain objects.",
  ]
},
{
  'name':'Technical Services',
  'responsibility':'connects to external devices and programs, e.g., for data persistence',
  'characteristics': [
      "Interacts with databases or other persistence services, e.g., inserts a row into a SQLite database.",
      "Performs network operations, e.g., sends a request to an email server.",
      "Interacts with the file system, e.g., saves an email attachment to disk.",
      "Uses Android's hardware-related APIs, e.g., checks if the device is connected to the internet.",
  ]
}]

In [ ]:
# print('In layered software architecture style, a software system is divided into 4 layers with different responsibilities:')
# print()
# for layer in layers:
#   print(f'- The {layer["name"]} layer {layer["responsibility"]}.')
# print()
# print('If we consider an Android app written in Java, here are 5 characteristics of Java methods that may indicate that the class they belong to resides in each of the above layer:')


# Presentation Layer

1. Interacts with Android's UI components, such as TextView, Button, RecyclerView, etc. For example, a method that sets the text of a TextView.
2. Handles user interactions, such as button clicks or list item selections. For example, a method annotated with @OnClick from the ButterKnife library.
3. Navigates between different screens (Activities or Fragments). For example, a method that starts a new Activity.
4. Updates the UI based on data changes. For example, a method that updates a RecyclerView when new data is loaded.
5. Shows or hides UI elements. For example, a method that shows a ProgressBar when a long-running task is started.

# Application Services Layer

1. Coordinates different parts of the application. For example, a method that starts a background task and then updates the UI when the task is finished.
2. Handles application lifecycle events. For example, a method that saves application state when the app is paused.
3. Handles errors. For example, a method that catches an exception and then shows an error message to the user.
4. Manages background tasks. For example, a method that starts a new Thread or uses an AsyncTask to perform a long-running task.
5. Interacts with Android's system services. For example, a method that uses the NotificationManager to show a notification.

# Domain Services Layer

1. Interacts with email-related objects. For example, a method that creates a new Email object.
2. Performs operations related to the email domain. For example, a method that sends an email or marks an email as read.
3. Validates email-related data. For example, a method that checks if an email address is valid.
4. Manages relationships between email-related objects. For example, a method that adds an Email to a Folder.
5. Handles email-related errors. For example, a method that catches an exception when an email can't be sent.

# Technical Services Layer

1. Interacts with databases. For example, a method that inserts a row into a SQLite database.
2. Performs network operations. For example, a method that sends a HTTP request to an email server.
3. Interacts with the file system. For example, a method that saves an email attachment to disk.
4. Uses Android's hardware-related APIs. For example, a method that checks if the device is connected to the internet.
5. Handles technical errors. For example, a method that catches an IOException when a network operation fails.

# Presentation Layer

- Characteristic 1: Interacts with Android's UI components, such as TextView, Button, RecyclerView, etc. For example, a method that sets the text of a TextView.
- Characteristic 2: Handles user interactions, such as button clicks or list item selections. For example, a method annotated with @OnClick from the ButterKnife library.
- Characteristic 3: Navigates between different screens (Activities or Fragments). For example, a method that starts a new Activity.
- Characteristic 4: Updates the UI based on data changes. For example, a method that updates a RecyclerView when new data is loaded.
- Characteristic 5: Shows or hides UI elements. For example, a method that shows a ProgressBar when a long-running task is started.

# Application Services Layer

- Characteristic 1: Coordinates different parts of the application. For example, a method that starts a background task and then updates the UI when the task is finished.
- Characteristic 2: Handles application lifecycle events. For example, a method that saves application state when the app is paused.
- Characteristic 3: Handles errors. For example, a method that catches an exception and then shows an error message to the user.

# Domain Services Layer

- Characteristic 1: Methods in this layer may be responsible for providing domain-specific functionality. For example, a method that composes various parts of an email message into a single structure.
- Characteristic 2: Methods in this layer may be responsible for validating domain-specific rules. For example, a method that checks if a piece of string is a valid email address.
- Characteristic 3: Methods in this layer may be responsible for transforming data. For example, a method that converts a list of messages into a list of message view models.
- Characteristic 4: Methods in this layer may be responsible for managing domain-specific state. For example, a method that updates the current user's email signature.
- Characteristic 5: Methods in this layer may be responsible for interacting with domain-specific external services. For example, a method that sends a payment to a payment gateway.

# Technical Services Layer

- Characteristic 1: Methods in this layer may be responsible for interacting with the file system. For example, a method that saves a file to disk.
- Characteristic 2: Methods in this layer may be responsible for interacting with the network. For example, a method that sends a HTTP request.
- Characteristic 3: Methods in this layer may be responsible for interacting with the database. For example, a method that queries the database for a list of products.
- Characteristic 4: Methods in this layer may be responsible for interacting with hardware devices. For example, a method that reads data from a sensor.
- Characteristic 5: Methods in this layer may be responsible for handling low-level details. For example, a method that encodes a string into a specific character set.

In [ ]:
layers = [
	{
		"name":"Presentation Layer",
		"class_characteristics": [
			"contains fields that represent UI objects such as buttons and text fields",
			"defines a UI markup template with markers of where to put data from model",
		],
		"method_characteristics": [
			"sets the attribute of UI objects based on information from other objects",
			"arranges UI objects in a particular layout",
			"observes changes in a data model and updates UI objects accordingly",
			"reacts to UI events by creating or invoking data model objects",
			"determines how to update a view according to the result of data model process result",
			"reacts to UI events by deciding what kind of action to initiate and delegates to a command to carry out the action ",
			"recognizes domain data and invokes the appropriate rendering transformations",
			"processes domain data element by element and transforms it into visual representation",
			"assembles model information in a presentation-oriented logical structure",
			"renders a presentation-oriented structure in a view markup", 
		]
	},
	{
		"name":"Service Layer",
		"class_characteristics": [
			"establishes a set of avaliable domain operations",
			"provides a coarse-grained interface that delegates to/coordinates finer-grained interfaces",
		],
		"method_characteristics": [
			"responds to user input, manipulates the model, and causes the view to update appropriately",
			"decides which domain logic to run upon an event",
			"decides the view with which to display the result of a domain operation",
			"encapsulates business logic ",
			"controls transactions and coordinates responses in the implementation of its operations",
			"wraps several calls to a domain model within one method",
		]
	},
	{
		"name":"Domain Layer",
		"class_characteristics": [
			"organizes business logic by procedures where each procedure handles a single request from the presentation",
			"handles the business logic for all rows in a database table or view",
			"groups data model with behavior that acts on that data",
			"represents a tabular data",
			"represents a serializable data",
			"consists of fields and getters and setters",
			"has simple or primitive fields",
		],
		"method_characteristics": [
			"makes calls to the data source",
			"represents some meaningful business rules or logic",
			"assembles information from multiple domain objects",
		]
	},
	{
		"name":"Data Source Layer",
		"class_characteristics": [
			"represents a single data record",
			"acts as a gateway to a data source",
			"has a simple interface consisting find, update, insert, and delete methods",
			"maintains a list of objects affected by a  transaction and coordinates the writing out of changes",
			"maintains an identity map to see which objects are already loaded from the data source",
			"wraps a record in a data source, encapsulates the data source access, and adds domain logic on that data",
			"is responsible for saving and loading to the data source and also for any domain logic that acts on the data",
		],
		"method_characteristics": [
			"performs conversion from data source-type to in-memory representation",
			"contains operations for insert/update/delete to a data source",
			"finds information from a data source and returns its representation",
			"returns a tabular data",
			"bundles changes to a data source as a single transaction",
			"performs inconsistency checking before commiting changes to a data source",
			"maps an object from the data source to an equivalent model object",
		]
	},
]

In [ ]:
layers

In [ ]:
for layer in layers:
  print(f"**{layer['name']} Layer** {layer['responsibility']}")
  for indicator in layer['characteristics']:
    print(f',"{indicator}"')

In [ ]:
def something() -> None:
  print('something')

something()

## Ask LLM to evaluate layer characteristics

In [ ]:
# goals = read_json_file(f"paper/{project_name}-goals_1.json")
# goals

In [ ]:
prompt_template = '''In layered software architecture, one of the layer is the {layer_name} Layer, which {layer_responsibility}.

Consider the context of an Android Java project "{project_name}": {project_desc}

Here are some indicators that a Java method in the project may belong to a class in the {layer_name} layer:

{layer_characteristics}

The {class_type} `{class_name}` contains the method `{method_name}`:

```java
{method_src}
```

Check whether this method satisfies each indicator above. Mention the specific line of code that supports your reason. At the very last line, write the boolean verdicts separated by comma, e.g.: `true, true, false, true`. If indeterminate, say `false`.'''

print(prompt_template)

In [ ]:
prompt_template = '''Consider the context of an Android Java project "{project_name}": {project_desc}.

The {class_type} `{class_name}` contains the method `{method_name}`:

```java
{method_src}
```

Which of the following statements could reflect the above method? There can be 0, 1, or more answers.

{method_characteristics}

Think step by step. At the end of your response, provide a line of boolean values, e.g., `true, false, true, ...` if statements 1 and 3 reflect the method in question.
'''

In [ ]:

num_methods = 0

samples = dict()

for pkg_name in sorted(list(packages)):

  classes = [ record['c'] for record in graph.run('MATCH (p:Container)-[:contains]->(c:Structure) '
                                                                   f'WHERE p.qualifiedName="{pkg_name}" AND p.kind="package" '
                                                                   'RETURN c') ]
  top_classes = [c for c in classes if not '$' in c['qualifiedName']]
  class_samples = random.sample(top_classes, min(len(top_classes),3))
  
  samples[pkg_name] = dict()
  for clss in class_samples:

    class_name = clss['qualifiedName']
    kind = clss['kind']
    methods = [ record['m'] for record in graph.run('MATCH (c:Structure)-[:hasScript]->(m:Operation) '
                                                    f'WHERE c.qualifiedName="{class_name}" AND m.visibility="public" AND m.kind="method" '
                                                    'return m') ]
    ok_methods = [m for m in methods if 300<len(m['sourceText'])<3000]
    method_samples = random.sample(ok_methods, min(len(ok_methods),10))
    # print(len(method_samples), [(clss['qualifiedName'], m['simpleName']) for m in method_samples])
    num_methods += len(method_samples)

    samples[pkg_name][class_name] = method_samples
print(samples)


In [ ]:
num_methods

In [ ]:
write_dict_to_json(samples, 'k9mail-samples.json')

In [ ]:
samples = read_json_file('k9mail-samples.json')
samples

In [ ]:
only_print_prompt = False

In [ ]:
results = dict()
kind = 'class'
for pkg_name in samples.keys():
  results[pkg_name] = dict()
  for class_name in samples[pkg_name].keys():
    results[pkg_name][class_name] = dict()
    for method in samples[pkg_name][class_name]:
      method_name = method['simpleName']
      results[pkg_name][class_name][method_name] = dict()
      print("#", pkg_name, class_name, method_name)
      print()
      print(method['sourceText'])
      print()
      for layer in layers:
        print(f'## {layer["name"]}')
        print()
        prompt = prompt_template.format(
            layer_name=layer["name"],
            layer_responsibility=layer["responsibility"],
            layer_characteristics="\n".join(
                [f"  {i+1}. {layer['characteristics'][i]}" for i in range(len(layer["characteristics"]))]),
            project_name=project_name,
            project_desc=project_desc,
            class_type=kind,
            class_name=class_name,
            method_name=method_name,
            # method_desc = goals[pkg_name]['classes'][row["c.qualifiedName"]]['methods'][method_name]['description'],
            method_src=remove_java_comments(method["sourceText"]))
        if only_print_prompt:
          print(prompt)
          print()
        else:
          response = None
          try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[{
                    "role": "user",
                    "content": prompt
                },{
                  "role": "assistant",
                  "content": "1."
                }],
                temperature=0)
            answer = response['choices'][0]['message']['content']
            results[pkg_name][class_name][method_name][layer['name']] = bools(answer.split('\n')[-1])
          except:
            answer = None
            print(response)
            results[pkg_name][class_name][method_name][layer['name']] = []
          print(answer)
          print()

      print(dict_to_pretty_json(results[pkg_name][class_name][method_name]))
      print()
      # break
    print(dict_to_pretty_json(results[pkg_name][class_name]))
    print()
    # break
  print(dict_to_pretty_json(results[pkg_name]))
  print()
  # break
print(dict_to_pretty_json(results))
    #         class_desc['methods'][method_name] = method_goal
    #   print("  " + str(goals[pkg_name]['classes'][class_name]))
    # print(str(goals[pkg_name]))
    # print()
# print(dict_to_pretty_json(goals))

print(num_methods)


In [ ]:
if not only_print_prompt:
  write_dict_to_json(results, f"{project_name}-layers_1.json")

In [ ]:
sum([len(clss) for pkg in results for clss in results[pkg]])

In [ ]:
def sum_tuples_by_index(input_list,key_idx,start_idx,end_idx):
    result_dict = {}

    for tup in input_list:
        key = tup[key_idx]
        if key in result_dict:
            # If the key is already in the dictionary, add the values element-wise
            result_dict[key] = tuple(sum(x) for x in zip(result_dict[key], tup[start_idx:end_idx]))
        else:
            # If the key is not in the dictionary, add it with the tuple as the value
            result_dict[key] = tup[start_idx:end_idx]

    # Convert the dictionary values back to a list of tuples
    result_list = list([(a,*b) for a,b in result_dict.items()])

    return result_list


# Test the function with your example input
input_list = [('a', 1, 0, 1), ('a', 0, 0, 1), ('b', 0, 1, 0), ('b', 0, 1, 1)]
result = sum_tuples_by_index(input_list, 0, 1, 4)
print(result)


In [ ]:
def find_index_of_max_value(input_list):
    if not input_list:
        # Handle the case where the list is empty
        return None

    max_value = input_list[0]
    max_index = 0

    for i in range(1, len(input_list)):
        if input_list[i] > max_value:
            max_value = input_list[i]
            max_index = i

    return max_index


# Test the function with your example list
my_list = [1, 0, 5, 3]
index_of_max = find_index_of_max_value(my_list)
print("Index of the largest value:", index_of_max)


In [ ]:
layer_names = [layer['name'] for layer in layers]
layer_names

In [ ]:
print('package,class,method,p1,p2,p3,p4,a1,a2,a3,a4,d1,d2,d3,d4,t1,t2,t3,t4,p,a,d,t')
tuples = []

for pkg in results:
  for clss in results[pkg]:
    for method in results[pkg][clss]:
      chs = []
      for layer in layers:
        ch = [0]*4
        # print(results[pkg][clss][method][layer['name']])
        for i,b in enumerate(results[pkg][clss][method][layer['name']]):
          ch[i] = 1 if b else 0
        chs.append(ch)
      sums = [1 if sum(row) else 0 for row in chs]
      # verdict = layer_names[find_index_of_max_value(sums)]
      tup = (pkg, clss, f'"{method}"', *[item for row in chs for item in row], *sums)
      print(*tup, sep=",")
      tuples.append(tup)

In [ ]:
class_layers = sum_tuples_by_index(tuples,1,3,19)
print('class,p1,p2,p3,p4,a1,a2,a3,a4,d1,d2,d3,d4,t1,t2,t3,t4')
for tup in class_layers:
  print(*tup, sep=",")


In [ ]:
class_layers = sum_tuples_by_index(tuples, 1, 19, 23)
print('class,p,a,d,t')
for tup in class_layers:
  print(*tup, sep=",")
